In [1]:
import cv2
import depthai as dai
import blobconverter
import numpy as np

In [2]:
num_of_classes = 21
nn_shape = 256

def decode_deeplabv3p(output_tensor):
    output = output_tensor.reshape(nn_shape,nn_shape)

    # scale to [0 ... 255] and apply colormap
    output = np.array(output) * (255/num_of_classes)
    output = output.astype(np.uint8)
    output_colors = cv2.applyColorMap(output, cv2.COLORMAP_TURBO)

    # reset the color of 0 class
    output_colors[output == 0] = [0,0,0]

    return output_colors

def show_deeplabv3p(output_colors, frame):
    return cv2.addWeighted(frame,1, output_colors,0.4,0)

## Pipeline erstellen

In [3]:
pipeline = dai.Pipeline()

## Nodes hinzufügen und verlinken

Im ersten Schritt erstellen wir eine Node für unsere RGB Kamera und konfigurieren diese für ein Segementation Neural Network. Wir setzen die Size auf die gleiche Größe wie der definierte Input des NN. Außerdem wird die Farbreihenfolge von RGB auf BGR umgestellt, da das NN diese Farbreihenfolge erwartet. Um das Programm performanter zu machen wird die Speicherung der Pixel von Interleaved (Bytes werden nach Pixel gespeichert) auf Planar (Bytes werden nach Farbe gespeichert) umgestellt.

In [4]:
rgb_cam = pipeline.createColorCamera()
rgb_cam.setPreviewSize(nn_shape,nn_shape)
rgb_cam.setInterleaved(False)
rgb_cam.setColorOrder(dai.ColorCameraProperties.ColorOrder.BGR)
rgb_cam.setFps(40)


Im nächsten Schritt erstellen wir eine Node für die Bildsegementierung. Dafür verwenden wir ein vortrainiertes Model vom Model Zoo. Es gibt eigens für DepthAI vortrainierte Modelle die man über den zoo_type auswählen kann. Der Parameter shaves gibt and wieviele der Cores für das NN auf der OAK-D verwendet werden sollen.

In diesem Fall verwenden wir das Model Deeplab und setzen ein paar zusätzliche Parameter wie wieviele Frames im Pool gespeichert werden sollen. Ob bei einer vollen Message Queue die ältesten Nachrichten gelöscht oder die Queue blockiert werden soll und wieiviel Threads das NN bekommen soll.

Am Ende verlinken wir den Output der vorher definierten Farbkamera mit der NN-Node als Input.

In [5]:
model_path = blobconverter.from_zoo(name="deeplab_v3_mnv2_256x256",
                                    zoo_type="depthai",
                                    shaves=6)
segmentation_nn = pipeline.create(dai.node.NeuralNetwork)
segmentation_nn.setBlobPath(model_path)
segmentation_nn.setNumPoolFrames(4)
segmentation_nn.input.setBlocking(False)
segmentation_nn.setNumInferenceThreads(2)
rgb_cam.preview.link(segmentation_nn.input)

Im nächsten Schritt erstellen wir ein weiteres NN zur Objektdetection von Gesichtern. Um genauer zu seien handelt es sich hierbei um eine eigene Node (MoblieNetDetectionnetwork). Der Unterschied zur NeuralNetwork Node ist der Output der Node. Ber der NN Node bekommen wir NNData als output bei der MobilenetDetectionNetwork Node bekommen wir Image Detections (label, confidence und boundingbox Koordinaten).

Da es sich hierbei aber auch um ein NN handelt müssen wir den Input darauf anpassen. Dafür kreiren wir eine weitere Node ImageManip. Diese dient dazu dass wir den Input der RGB Kamera so manipulieren, dass er für das neuronale Netz geeignet ist.

In [6]:
face_model = blobconverter.from_zoo(name="face-detection-retail-0004",
                                        zoo_type="depthai",
                                        shaves=6)
face_nn = pipeline.create(dai.node.MobileNetDetectionNetwork)
face_nn.setBlobPath(face_model)
face_manip = pipeline.createImageManip()
face_manip.initialConfig.setResize(300,300)
face_manip.initialConfig.setKeepAspectRatio(False)

In diesem Schritt linken wir nun den Input der RGB Kamera der auf das erste Netz angepasst ist mit der ImageManip Node, damit wir den gleichen Input für das zweite NN verwenden können. Dann wird der Output vom ImageManip mit dem Input der Gesichtserkennungs-Node verknüpft.

In [7]:
rgb_cam.preview.link(face_manip.inputImage)
face_manip.out.link(face_nn.input)

Das XLinkout und setStreamName ist der gleiche Vorgang wie im getting_started Beispiel. Neu hinzugekommen ist passthrough. Mit passthrough haben wir die Möglichkeit den Input in ein NN direkt weiterzuleiten damit dieser in einem anderen Schritt wiederverwendet werden kann.

In diesem Beispiel verwenden wir den passthrough dafür, dass wir das RGB Bild mit XLinkOut am Laptop wiedergeben können.

In [8]:
xout_rgb = pipeline.createXLinkOut()
xout_rgb.setStreamName("rgb")
xout_rgb.input.setBlocking(False)


segmentation_nn.passthrough.link(xout_rgb.input)

In den nächsten zwei Schritten werden dei XLinkOut für die zwei NN erstellt.

In [9]:
xout_nn = pipeline.create(dai.node.XLinkOut)
xout_nn.setStreamName("nn")
xout_nn.input.setBlocking(False)
segmentation_nn.out.link(xout_nn.input)

In [10]:
xout_face = pipeline.create(dai.node.XLinkOut)
xout_face.setStreamName("face")
face_nn.out.link(xout_face.input)

## Pipeline laufen lassen
Wie im vorgegangenen Beispiel lassen wir die Pipeline nun laufen. Wir kreieren drei Outputstreams (RGB Kamera, Objeksegmentierung und Gesichtserkennung). Lesen von der Gesichtserkennung die Boundingbox aus in dem wir auf die detections des Gesichtserkennungsstreams zugreifen und lassen sie mit cv2.rectangle() zeichnen.

Zusätzlich greifen wir auf die Layer des NN zur Objektsegementierung zu und decodieren die Informationen damit wir die Segmentierung über das Bild legen können.

Am Ende vergrößern wir das Bild wieder auf 600x600 mit den opencv tool resize.

In [11]:
with dai.Device() as device:
    device.startPipeline(pipeline)
    cam_queue = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    nn_queue = device.getOutputQueue(name="nn", maxSize=4, blocking=False)
    face_queue = device.getOutputQueue(name="face", maxSize=1, blocking=False)

    while True:

        bbox = None

        detection_frame = face_queue.tryGet()

        if detection_frame is not None:
            detections = detection_frame.detections
            if len(detections) > 0 and len(detections) is not None:
                print(detections[0])
                detection = detections[0]

                xmin = max(0, detection.xmin)
                ymin = max(0, detection.ymin)
                xmax = min(detection.xmax, 1)
                ymax = min(detection.ymax, 1)

                x = int(xmin*600)
                y = int(ymin*600)
                w = int(xmax*600-xmin*600)
                h = int(ymax*600-ymin*600)

                bbox = (x,y,w,h)



        color_frame = cam_queue.get()
        color_frame = color_frame.getCvFrame()
        nn_frame = nn_queue.get()
        nn_layers = nn_frame.getAllLayers()
        
        layer_1 = np.array(nn_frame.getFirstLayerInt32()).reshape(nn_shape,nn_shape)
        found_classes = np.unique(layer_1)
        output_colors = decode_deeplabv3p(layer_1)

        frame = show_deeplabv3p(output_colors, color_frame)
        frame = cv2.resize(frame, (600,600))
        cv2.rectangle(frame, bbox, (0, 255, 0), 2 )
        cv2.putText(frame, "Found classes {}".format(found_classes), (2, 10), cv2.FONT_HERSHEY_TRIPLEX, 0.4, (255, 0, 0))
        cv2.imshow("RGB", frame)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

cv2.destroyAllWindows

<function destroyAllWindows>

: 